In [24]:
import json

In [ ]:
# Some constants to work with
FILE_WITH_PIPELINE = "<json file with pipeline code>"
PLACEHOLDER_FOR_NONE = "None"
PLACEHOLDER_FOR_EMPTY = "(empty)"
DOC_TEMPLATE = """[[_TOC_]]
# Parameters

<Parameters>

# Variables

<Variables>

# Pipeline
"""
TABLE_TEMPLATE = """|#|Name|Type|Default value|
|-|-|-|-|
"""
MERMAID_DIAGRAM_TEMPLATE = """:::mermaid
graph LR
<Graph>
:::"""

In [ ]:
# Read JSON file with pipeline code
with open(FILE_WITH_PIPELINE, "r") as f:
    content = f.read()
    data = json.loads(content)

In [ ]:
# Get some objects from code
title = data["name"]
properties = data["properties"]
parameters = properties["parameters"]
variables = properties["variables"]
pipeline_description = properties["description"] if "description" in properties.keys() else None # not in use
folder = properties["folder"] # not in use
annotations = properties["annotations"] # not in use

In [ ]:
# Process parameters
if parameters:
    parameters_table = TABLE_TEMPLATE
    i = 1
    for v in parameters:
        parameters_table += f"|{i}|{v}|{parameters[v]["type"].title()}|{parameters[v]["defaultValue"] if "defaultValue" in parameters[v] else PLACEHOLDER_FOR_EMPTY}|\n"
        i += 1
else:
    PLACEHOLDER_FOR_NONE

In [ ]:
# Process variables
if variables:
    variables_table = TABLE_TEMPLATE
    i = 1
    for v in variables:
        variables_table += f"|{i}|{v}|{variables[v]["type"].title()}|{variables[v]["defaultValue"] if "defaultValue" in variables[v].keys() else PLACEHOLDER_FOR_EMPTY}|\n"
        i += 1
else:
    PLACEHOLDER_FOR_NONE

In [30]:
# Function to get all activities in code
def find_activities(properties:dict, parent:str="Main", results:list=None):
    if results is None:
        results = []

    if isinstance(properties, dict):
        for key, value in properties.items():
            if key == 'activities':
                results.append({"parent": parent, "values": value})
            if key == "name":
                parent=value
            find_activities(value, parent, results)

    elif isinstance(properties, list):
        for item in properties:
            find_activities(item, parent, results)

    return results

In [39]:
# Function to parse activities. It creates diagram and sorted list of activities
def parse_activities(activities:list):
    activity_titles = []
    activity_descriptions = []
    apexes = []
    edges = []
    for a in activities:
        # Prepare components for diagram
        a_pretty_name = a["name"]
        a_formal_name = a_pretty_name.title().replace(" ", "")
        apexes.append(f"{a_formal_name}[{a_pretty_name}]")

        a_description = a["description"] if "description" in a.keys() else PLACEHOLDER_FOR_NONE

        predecessor_position = 0
        for d in a["dependsOn"]:
            d_formal_name = d["activity"].title().replace(" ", "")
            arrow_comment = ", ".join(d["dependencyConditions"])
            edges.append(f"{d_formal_name} --> |{arrow_comment}| {a_formal_name}")
            # Sort activities within one dag
            if d["activity"] in activity_titles:
                predecessor_position = activity_titles.index(d["activity"]) if activity_titles.index(d["activity"]) > predecessor_position else predecessor_position
        activity_titles.insert(predecessor_position + 1, a_pretty_name)
        activity_descriptions.insert(predecessor_position + 1, a_description)

    diagram = MERMAID_DIAGRAM_TEMPLATE.replace("<Graph>", f"{"\n".join(apexes)}\n{"\n".join(edges)}")
    return list(zip(activity_titles, activity_descriptions)), diagram

In [ ]:
# Process activities
all_activities = {}
all_diagrams = []
for activities in find_activities(properties):
    diagram_title = f"## _{activities["parent"]}_"
    tuples, diagram = parse_activities(activities["values"])
    all_diagrams.append(f"{diagram_title}\n{diagram}")
    all_activities[activities["parent"]] = tuples

In [ ]:
# Function to structure activities properly (sorted and with proper intense)
structured_activities = []
def structure_activities(parent:str, i:int):
    for x in all_activities[parent]:
        structured_activities.append(f"{'    ' * i}1. _{x[0]}_ - {x[1]}")
        if x[0] in all_activities.keys():
            structure_activities(x[0], i+1)
    return structured_activities

structure_activities("Main", 0)

In [ ]:
# Save result as Markdown file
result = f"""{title.upper()}

{DOC_TEMPLATE.replace('<Parameters>', parameters_table).replace('<Variables>', variables_table)}{"\n".join(all_diagrams)}
{'\n'.join(structured_activities)}
"""
with open("test.md", "w") as f:
    f.write(result)